In [ ]:
# 10 cm depth soil moisture

import ee
import geemap
import pandas as pd
import datetime
import os

# Authenticate and initialize EE
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location (Lankien, South Sudan)
lankien = ee.Geometry.Point([33.5669, 8.4354])

# Define date range
start_date = '2015-12-01'
end_date = datetime.datetime.today().strftime('%Y-%m-%d')

# Load GLDAS-2 NOAH dataset
gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
    .filterBounds(lankien) \
    .filterDate(start_date, end_date)

# Function to extract soil moisture
def extract_soil_moisture(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    soil_moisture = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=lankien,
        scale=10000  # 10 km resolution
    ).get('SoilMoi0_10cm_inst')  # Soil moisture at 0-10 cm depth
    
    return ee.Feature(lankien, {'date': date, 'soil_moisture': soil_moisture})

# Apply function and convert to FeatureCollection
features = gldas.map(extract_soil_moisture)
feature_collection = ee.FeatureCollection(features)  # Explicit conversion

# Convert EE FeatureCollection to Pandas DataFrame
df = geemap.ee_to_df(feature_collection)


# Save to CSV

output_folder = "Soil_Moisture"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "soil_moisture_gldas_lankien.csv")
df.to_csv(output_path, index=False)

# Display results
print("GLDAS Soil Moisture Data (First 5 rows):")
print(df.head())


In [1]:
import ee
import pandas as pd
import os
import datetime

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location (Lankien, South Sudan)
latitude, longitude = 8.4281, 33.7833
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = datetime.date.today().strftime('%Y-%m-%d')  # Get today's date in 'YYYY-MM-DD' format

# Soil Moisture from SMAP
soil_moisture_collection = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to extract soil moisture values
def extract_soil_moisture(image):
    date = image.date().format("YYYY-MM-dd")
    soil_moisture_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=5000,  # SMAP has 5 km resolution
        bestEffort=True
    ).get('ssm')  # 'ssm' is the band for soil moisture
    return ee.Feature(None, {'date': date, 'soil_moisture': soil_moisture_value})

# Extract soil moisture
soil_moisture_features = soil_moisture_collection.map(extract_soil_moisture).getInfo()

# Combine the extracted features into a dataframe
data = []
for feature in soil_moisture_features['features']:
    date = feature['properties']['date']
    soil_moisture = feature['properties']['soil_moisture']
    data.append([date, soil_moisture])

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=['Date', 'Soil_Moisture'])

# Save as CSV
output_folder = "Soil_Moisture"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "Soil_Moisture_Lankien21.csv")

df.to_csv(output_path, index=False)
print(f"Soil moisture data saved to {output_path}")


/Users/berhe/Desktop/ka-project/.venv/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for NASA_USDA/HSL/SMAP_soil_moisture! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_SMAP_soil_moisture

  warnings.warn(warning, category=DeprecationWarning)


Soil moisture data saved to Soil_Moisture/Soil_Moisture_Lankien21.csv


# Daily

In [2]:
import ee
import pandas as pd
import os
import datetime

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location (Lankien, South Sudan)
latitude, longitude = 8.4281, 33.7833
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = datetime.date.today().strftime('%Y-%m-%d')  # Get today's date in 'YYYY-MM-DD' format

# Soil Moisture from SMAP (daily data)
soil_moisture_collection = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to extract daily soil moisture values
def extract_soil_moisture(image):
    date = image.date().format("YYYY-MM-dd")  # Format the date
    soil_moisture_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=5000,  # SMAP has 5 km resolution
        bestEffort=True
    ).get('ssm')  # 'ssm' is the band for soil moisture
    return ee.Feature(None, {'date': date, 'soil_moisture': soil_moisture_value})

# Extract daily soil moisture values
soil_moisture_features = soil_moisture_collection.map(extract_soil_moisture).getInfo()

# Combine the extracted features into a dataframe
data = []
for feature in soil_moisture_features['features']:
    date = feature['properties']['date']
    soil_moisture = feature['properties']['soil_moisture']
    data.append([date, soil_moisture])

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=['Date', 'Soil_Moisture'])

# Save as CSV
output_folder = "Soil_Moisture"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "NASA_Daily_Soil_Moisture_Lankien.csv")

df.to_csv(output_path, index=False)
print(f"Daily soil moisture data saved to {output_path}")


Daily soil moisture data saved to Soil_Moisture/NASA_Daily_Soil_Moisture_Lankien.csv


# Soil Moisture after 2020

In [3]:
import ee
import pandas as pd
import os
import datetime

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location (Lankien, South Sudan)
latitude, longitude = 8.4281, 33.7833
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = datetime.date.today().strftime('%Y-%m-%d')  # Get today's date in 'YYYY-MM-DD' format

# Soil Moisture from SMAP (daily data)
soil_moisture_esa = ee.ImageCollection('ESA/CCI/SM') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to extract daily soil moisture values
def extract_soil_moisture(image):
    date = image.date().format("YYYY-MM-dd")  # Format the date
    soil_moisture_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=5000,  # SMAP has 5 km resolution
        bestEffort=True
    ).get('ssm')  # 'ssm' is the band for soil moisture
    return ee.Feature(None, {'date': date, 'soil_moisture': soil_moisture_value})

# Extract daily soil moisture values
soil_moisture_features = soil_moisture_collection.map(extract_soil_moisture).getInfo()

# Combine the extracted features into a dataframe
data = []
for feature in soil_moisture_features['features']:
    date = feature['properties']['date']
    soil_moisture = feature['properties']['soil_moisture']
    data.append([date, soil_moisture])

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=['Date', 'Soil_Moisture'])

# Save as CSV
output_folder = "Soil_Moisture"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "ESA_Daily_Soil_Moisture_Lankien.csv")

df.to_csv(output_path, index=False)
print(f"Daily soil moisture data saved to {output_path}")

Daily soil moisture data saved to Soil_Moisture/ESA_Daily_Soil_Moisture_Lankien.csv


In [4]:
import ee
import pandas as pd
import os
import datetime

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location (Lankien, South Sudan)
latitude, longitude = 8.4281, 33.7833
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = datetime.date.today().strftime('%Y-%m-%d')  # Get today's date in 'YYYY-MM-DD' format

# Soil Moisture from SMAP (daily data)
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A1') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to extract daily soil moisture values
def extract_soil_moisture(image):
    date = image.date().format("YYYY-MM-dd")  # Format the date
    soil_moisture_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=5000,  # SMAP has 5 km resolution
        bestEffort=True
    ).get('ssm')  # 'ssm' is the band for soil moisture
    return ee.Feature(None, {'date': date, 'soil_moisture': soil_moisture_value})

# Extract daily soil moisture values
soil_moisture_features = soil_moisture_collection.map(extract_soil_moisture).getInfo()

# Combine the extracted features into a dataframe
data = []
for feature in soil_moisture_features['features']:
    date = feature['properties']['date']
    soil_moisture = feature['properties']['soil_moisture']
    data.append([date, soil_moisture])

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=['Date', 'Soil_Moisture'])

# Save as CSV
output_folder = "Soil_Moisture"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "MODIS_Daily_Soil_Moisture_Lankien.csv")

df.to_csv(output_path, index=False)
print(f"Daily soil moisture data saved to {output_path}")

Daily soil moisture data saved to Soil_Moisture/MODIS_Daily_Soil_Moisture_Lankien.csv


In [5]:
import ee
import pandas as pd
import os
import datetime

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location (Lankien, South Sudan)
latitude, longitude = 8.4281, 33.7833
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = datetime.date.today().strftime('%Y-%m-%d')  # Get today's date in 'YYYY-MM-DD' format

# Soil Moisture from SMAP (daily data)
soil_moisture_collection = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to extract daily soil moisture values
def extract_soil_moisture(image):
    date = image.date().format("YYYY-MM-dd")  # Format the date
    soil_moisture_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=36000,  # SMAP has 36 km resolution
        bestEffort=True
    ).get('ssm')  # 'ssm' is the band for soil moisture
    return ee.Feature(None, {'date': date, 'soil_moisture': soil_moisture_value})

# Extract daily soil moisture values
soil_moisture_features = soil_moisture_collection.map(extract_soil_moisture).getInfo()

# Combine the extracted features into a dataframe
data = []
for feature in soil_moisture_features['features']:
    date = feature['properties']['date']
    soil_moisture = feature['properties']['soil_moisture']
    data.append([date, soil_moisture])

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=['Date', 'Soil_Moisture'])

# Save as CSV
output_folder = "Soil_Moisture"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "NASA_Daily_Soil_Moisture_Lankien.csv")

df.to_csv(output_path, index=False)
print(f"Daily soil moisture data saved to {output_path}")


Daily soil moisture data saved to Soil_Moisture/NASA_Daily_Soil_Moisture_Lankien.csv


# Moisture from Copernicus to include data from 2020 onwards as well 

In [ ]:
import cdsapi

c = cdsapi.Client()

# Define the range of years and months
years = [str(y) for y in range(2015)]  # Update 2025 to current year if needed
months = [str(m).zfill(2) for m in range(1, 13)]  # '01' to '12'

for year in years:
    for month in months:
        print(f"Requesting data for {year}-{month}...")
        c.retrieve(
            'satellite-soil-moisture',
            {
                'variable': 'surface_soil_moisture',
                'format': 'netcdf',
                'time_aggregation': 'daily',
                'year': year,
                'month': month,
                'day': [str(i).zfill(2) for i in range(1, 32)],  # All days
                'version': 'v202012',
                'sensor': 'scatterometer',
                'product_type': 'passive',
                'area': [30.957, 9.205, 32.008, 7.480],  # [North, West, South, East] Example: Africa
            },
            f"soil_moisture_{year}_{month}.nc"
        )


In [ ]:
import cdsapi

dataset = "satellite-soil-moisture"
request = {
    "variable": ["surface_soil_moisture"],
    "type_of_sensor": ["active"],
    "time_aggregation": ["day_average"],
    "year": ["2021", "2022"],
    "month": [
        "01", "02", "03", "04", "05", "06",
        "07", "08", "09", "10", "11", "12"
    ],
    "day": [
        "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12",
        "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", 
        "25", "26", "27", "28", "29", "30", "31"
    ],
    "type_of_record": [
        "icdr", "cdr"
    ],
    "version": [
        "v202312", "deprecated_v201912", "v201812", "v201912_1", "v202012", "v202212"
    ]
}

client = cdsapi.Client()

# Download the data
client.retrieve(dataset, request).download("soil_moisture_2021_2022.zip")


In [7]:
import ee
import geemap
import numpy as np
import pandas as pd
import os

# Authenticate and Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define the study area (Lankien, South Sudan)
lankien_bbox = ee.Geometry.Rectangle([30.957, 7.480, 32.008, 9.205])

# Load SMAP L3 Soil Moisture Data (Daily, 9km resolution)
dataset = ee.ImageCollection("NASA_USDA/HSL/SMAP_soil_moisture") \
    .filterBounds(lankien_bbox) \
    .filterDate("2015-11-01", "2025-01-01") \
    .select("ssm")  # Surface Soil Moisture (m³/m³)

# Function to extract soil moisture values
def extract_soil_moisture(image):
    sm_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=lankien_bbox,
        scale=9000,  # 9km resolution
        bestEffort=True,
        maxPixels=1e13
    ).get("ssm")

    return ee.Feature(None, {
        "Soil_Moisture": sm_value,
        "date": image.date().format("YYYY-MM-dd")
    })

# Convert ImageCollection to FeatureCollection
soil_moisture_features = dataset.map(extract_soil_moisture)

# Extract values as lists
sm_list = soil_moisture_features.aggregate_array("Soil_Moisture").getInfo()
dates = soil_moisture_features.aggregate_array("date").getInfo()

# Convert to Pandas DataFrame
df_soil_moisture = pd.DataFrame({"date": dates, "Soil_Moisture": sm_list})
df_soil_moisture["date"] = pd.to_datetime(df_soil_moisture["date"])
df_soil_moisture = df_soil_moisture.dropna()  # Remove missing values

# Save DataFrame as CSV
output_folder = "SoilMoisture"
os.makedirs(output_folder, exist_ok=True)

df_soil_moisture.to_csv(os.path.join(output_folder, "SoilMoisture_Lankien.csv"), index=False)

# Print first few rows
print("Daily Soil Moisture Data (m³/m³):")
print(df_soil_moisture.head())


Daily Soil Moisture Data (m³/m³):
        date  Soil_Moisture
0 2015-11-01      20.415980
1 2015-11-04      19.933183
2 2015-11-07      17.728078
3 2015-11-10      16.572906
4 2015-11-13      15.424095


In [25]:
import ee

# Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Load the SMAP soil moisture dataset
dataset = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture")
print("Total images in dataset:", dataset.size().getInfo())

# Get the date of the first image in the collection
first_image = dataset.first()
date_range = first_image.date().format("YYYY-MM-dd").getInfo()
print("First Image Date:", date_range)

# Define time range for filtering (modify as needed)
start_date = "2021-01-01"
end_date = "2025-01-01"

# Filter the collection by date
filtered_collection = dataset.filterDate(start_date, end_date)
print("Number of images in selected period:", filtered_collection.size().getInfo())

# Compute mean soil moisture over the selected period
mean_soil_moisture = filtered_collection.mean()

# Define a point location (Latitude, Longitude) for data extraction
point = ee.Geometry.Point([33.7833, 8.428])  

# Extract soil moisture at the point
soil_moisture_at_point = mean_soil_moisture.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=point,
    scale=10000  # Adjust scale based on resolution
).getInfo()

print("Soil Moisture at Point:", soil_moisture_at_point)


Total images in dataset: 894
First Image Date: 2015-04-02
Number of images in selected period: 193
Soil Moisture at Point: {'smp': 0.40052303671836853, 'ssm': 9.702286720275879, 'ssma': -0.29835477471351624, 'susm': 84.82115173339844, 'susma': -0.2605511248111725}


In [28]:
import ee
import pandas as pd

# Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Load the SMAP soil moisture dataset
dataset = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture")
print("Total images in dataset:", dataset.size().getInfo())

# Define time range for filtering
start_date = "2015-11-01"
end_date = "2025-01-01"

# Filter the collection by date
filtered_collection = dataset.filterDate(start_date, end_date)
print("Number of images in selected period:", filtered_collection.size().getInfo())

# Define a point location (Latitude, Longitude)
point = ee.Geometry.Point([33.7833, 8.428])

# Function to extract soil moisture values per image
def extract_soil_moisture(image):
    # Reduce region at the point
    value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=10000
    ).get("ssm")  # 'ssm' = Surface Soil Moisture Band
    
    # Get date of the image
    date = image.date().format("YYYY-MM-dd")
    
    # Return feature with soil moisture and date
    return ee.Feature(None, {"date": date, "soil_moisture": value})

# Apply function to collection
soil_moisture_features = filtered_collection.map(extract_soil_moisture)

# Extract values as lists
sm_list = soil_moisture_features.aggregate_array("soil_moisture").getInfo()
dates = soil_moisture_features.aggregate_array("date").getInfo()

# Convert to Pandas DataFrame
df = pd.DataFrame({"Date": dates, "Soil_Moisture": sm_list})

# Save as CSV
csv_filename = "soil_moisture_data.csv"
df.to_csv(csv_filename, index=False)

print(f"✅ Data saved to {csv_filename}")


Total images in dataset: 894
Number of images in selected period: 823
✅ Data saved to soil_moisture_data.csv
